In [ ]:
# imports 
import os
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import tensorflow as tf
from collections import Counter
import random
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelBinarizer

# Read data

In [2]:
# Define Paths
dataset_dir = "Data"  # Root folder containing class folders
# Adjust class names to match actual folder names
classes = ["Mild Dementia", "Moderate Dementia", "Non Demented", "Very mild Dementia"]

# Load Data with Correct Folder Names
image_paths, labels = [], []
for class_label, class_name in enumerate(classes):
    class_dir = os.path.join(dataset_dir, class_name)
    if not os.path.exists(class_dir):
        print(f"Error: Folder {class_dir} does not exist.")
        continue
    files = glob.glob(f"{class_dir}/*.jpg")  # Adjust extension if needed
    print(f"Class: {class_name}, Files Found: {len(files)}")  # Debug: Count files
    for file_path in files:
        image_paths.append(file_path)
        labels.append(class_label)

# Proceed with the pipeline if files are found
if len(image_paths) == 0:
    raise ValueError("No images found. Check dataset folder names or file paths.")

Class: Mild Dementia, Files Found: 5002
Class: Moderate Dementia, Files Found: 488
Class: Non Demented, Files Found: 67222
Class: Very mild Dementia, Files Found: 13725


In [3]:
# Split data into training, testing and validation
train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.3, random_state=42,stratify=labels) # suffle by default and straity labels 
test_paths, val_paths, test_labels, val_labels = train_test_split(test_paths, test_labels, test_size=0.5, random_state=42,stratify=test_labels) # suffle by default and straity labels
# to keep the same class distribution

In [4]:
# Count classes in each split
train_class_counts = Counter(train_labels)
test_class_counts = Counter(test_labels)
val_class_counts = Counter(val_labels)

print("Training set class distribution:", train_class_counts)
print("Testing set class distribution:", test_class_counts)
print("Validation set class distribution:", val_class_counts)

Training set class distribution: Counter({2: 47055, 3: 9607, 0: 3501, 1: 342})
Testing set class distribution: Counter({2: 10084, 3: 2059, 0: 750, 1: 73})
Validation set class distribution: Counter({2: 10083, 3: 2059, 0: 751, 1: 73})


In [11]:

train_paths = np.array(train_paths)
test_paths = np.array(test_paths)
val_paths = np.array(val_paths)

# One-hot encode the labels
label_binarizer = LabelBinarizer()
train_labels = label_binarizer.fit_transform(train_labels)
test_labels = label_binarizer.transform(test_labels)
val_labels = label_binarizer.transform(val_labels)

# Function to augment a single image
def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    image = tf.image.random_hue(image, max_delta=0.1)
    return image, label

# Function to apply augmentation multiple times
def augment_multiple_times(image_path, label, num_times):
    dataset = tf.data.Dataset.from_tensors((image_path, label))  # Original image 
    augmented_images = dataset
         
    for _ in range(num_times):
        augmented_images = augmented_images.concatenate(
            tf.data.Dataset.from_tensors((image_path, label)).map(augment_image)
        )
        
    return augmented_images

def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    # Normalize (scale to [0,1])
    image = tf.image.convert_image_dtype(image, tf.float32)  
    return image

# Load Dataset
def load_dataset(image_paths, labels):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda x, y: (preprocess_image(x), y))
    return dataset

# Apply preprocessing so all datasets contain image tensors (not paths)
preprocessed_train_dataset = load_dataset(train_paths, train_labels)
preprocessed_test_dataset = load_dataset(test_paths, test_labels)
preprocessed_val_dataset = load_dataset(val_paths, val_labels)

# Split by class
class_2_train_dataset = preprocessed_train_dataset.filter(lambda img, label: tf.equal(tf.argmax(label), 2)).take(10_000)
class_3_train_dataset = preprocessed_train_dataset.filter(lambda img, label: tf.equal(tf.argmax(label), 3))
class_0_train_dataset = preprocessed_train_dataset.filter(lambda img, label: tf.equal(tf.argmax(label), 0))
class_1_train_dataset = preprocessed_train_dataset.filter(lambda img, label: tf.equal(tf.argmax(label), 1))

# Apply augmentation correctly
class_0_train_augmented = class_0_train_dataset.flat_map(lambda img, lbl: augment_multiple_times(img, lbl, 1))
class_1_train_augmented = class_1_train_dataset.flat_map(lambda img, lbl: augment_multiple_times(img, lbl, 9))

# Combine the datasets
final_train_dataset = (class_2_train_dataset
                 .concatenate(class_3_train_dataset)
                 .concatenate(class_0_train_augmented)
                 .concatenate(class_1_train_augmented))



#{2: 10083, 3: 2059, 0: 751, 1: 73}
# Split by class
class_2_val_dataset = preprocessed_val_dataset.filter(lambda img, label: tf.equal(tf.argmax(label), 2)).take(10_000)
class_3_val_dataset = preprocessed_val_dataset.filter(lambda img, label: tf.equal(tf.argmax(label), 3))
class_0_val_dataset = preprocessed_val_dataset.filter(lambda img, label: tf.equal(tf.argmax(label), 0))
class_1_val_dataset = preprocessed_val_dataset.filter(lambda img, label: tf.equal(tf.argmax(label), 1))

# Apply augmentation correctly
class_0_val_dataset_augmented = class_0_val_dataset.flat_map(lambda img, lbl: augment_multiple_times(img, lbl, 8))
class_1_val_dataset_augmented = class_1_val_dataset.flat_map(lambda img, lbl: augment_multiple_times(img, lbl, 40))
class_3_val_dataset_augmented = class_3_val_dataset.flat_map(lambda img, lbl: augment_multiple_times(img, lbl, 3))

# Combine the datasets
final_val_dataset = (class_2_val_dataset
                 .concatenate(class_3_val_dataset_augmented)
                 .concatenate(class_0_val_dataset_augmented)
                 .concatenate(class_1_val_dataset_augmented))



final_test_dataset = preprocessed_test_dataset

In [12]:
label_counts = Counter()

# Loop through dataset
for _, lbl in final_train_dataset:
    class_index = np.argmax(lbl.numpy())  # Convert one-hot to class index
    label_counts[class_index] += 1

# Print class distribution
print("Final dataset class distribution:")
for label, count in sorted(label_counts.items()):
    print(f"Class {label}: {count} images")


2025-04-07 16:38:05.460397: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype string and shape [60505]
	 [[{{node Placeholder/_7}}]]


Final dataset class distribution:
Class 0: 7002 images
Class 1: 3420 images
Class 2: 10000 images
Class 3: 9607 images


In [13]:
label_val_counts = Counter()

# Loop through dataset
for _, lbl in final_val_dataset:
    class_index = np.argmax(lbl.numpy())  # Convert one-hot to class index
    label_val_counts[class_index] += 1

# Print class distribution
print("Final val dataset class distribution:")
for label, count in sorted(label_val_counts.items()):
    print(f"Class {label}: {count} images")


2025-04-07 16:38:42.437528: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_21' with dtype string and shape [12966]
	 [[{{node Placeholder/_21}}]]


Final val dataset class distribution:
Class 0: 6759 images
Class 1: 2993 images
Class 2: 10000 images
Class 3: 8236 images


In [14]:
# Define the CNN model
def create_cnn(num_classes=4):
    model = keras.Sequential([
        # Convolutional Block 1
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2,2)),
        layers.BatchNormalization(),
        
        # Convolutional Block 2
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.BatchNormalization(),
        
        # Convolutional Block 3
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.BatchNormalization(),
        
        # Flatten & Dense Layers
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),  # Reduce overfitting
        layers.Dense(num_classes, activation='softmax')  # Output layer

        # layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        # layers.MaxPooling2D((2, 2)),
        # layers.Conv2D(64, (3, 3), activation='relu'),
        # layers.MaxPooling2D((2, 2)),
        # layers.Conv2D(128, (3, 3), activation='relu'),
        # layers.MaxPooling2D((2, 2)),
        # layers.Flatten(),
        # layers.Dense(128, activation='relu'),
        # layers.Dropout(0.5),
        # layers.Dense(num_classes, activation='softmax')  # Number of classes


        # tf.keras.layers.Conv2D(64,kernel_size=(5,5), input_shape = (224,224,3), activation='relu'),
        # tf.keras.layers.Conv2D(64,kernel_size=(5,5), activation='relu'),
        # #tf.keras.layers.BatchNormalization(),
        # tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        # tf.keras.layers.Dropout(0.01),

        # tf.keras.layers.Conv2D(64,kernel_size=(5,5),activation='relu'),
        # tf.keras.layers.Conv2D(64,kernel_size=(5,5),activation='relu'),
        # #tf.keras.layers.BatchNormalization(),
        # tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        # tf.keras.layers.Dropout(0.01),

        # tf.keras.layers.Conv2D(64,kernel_size=(5,5),activation='relu'),
        # tf.keras.layers.Conv2D(64,kernel_size=(5,5),activation='relu'),
        # #tf.keras.layers.BatchNormalization(),
        # tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        # tf.keras.layers.Dropout(0.01),

        # tf.keras.layers.Conv2D(64,kernel_size=(5,5),activation='relu'),
        # tf.keras.layers.Conv2D(64,kernel_size=(5,5),activation='relu'),
        # #tf.keras.layers.BatchNormalization(),
        # tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        # tf.keras.layers.Dropout(0.01),
        
        # #tf.keras.layers.Conv2D(64,kernel_size=(5,5),activation='relu'),
        # #tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        # #tf.keras.layers.Dropout(0.01),
        
        # tf.keras.layers.Flatten(),
        # tf.keras.layers.Dense(4,activation='softmax')
    ])

    # Compile the model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model



In [15]:
# Create the model
cnn_model = create_cnn()

# Print model summary
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 111, 111, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                        

In [16]:
print("Test dataset shape:", final_test_dataset)
print("Final dataset shape:", final_train_dataset)
print("Test dataset shape:", final_val_dataset)

Test dataset shape: <_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4,), dtype=tf.int64, name=None))>
Final dataset shape: <_ConcatenateDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4,), dtype=tf.int64, name=None))>
Test dataset shape: <_ConcatenateDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4,), dtype=tf.int64, name=None))>


In [17]:
BATCH_SIZE = 32

# Batch the dataset
train_dataset = final_train_dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = final_val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

cnn_model.fit(train_dataset, validation_data=val_dataset, epochs=10)
#cnn_model.fit(final_dataset, epochs=10, steps_per_epoch=len(final_dataset))

Epoch 1/10


2025-04-07 16:39:48.022151: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_20' with dtype string and shape [60505]
	 [[{{node Placeholder/_20}}]]
2025-04-07 16:39:48.022439: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [60505]
	 [[{{node Placeholder/_0}}]]


     52/Unknown - 24s 446ms/step - loss: 0.0922 - accuracy: 0.9814

KeyboardInterrupt: 

In [ ]:
for img, label in final_dataset.take(1):
    print(img.shape, label.shape)


In [ ]:
for img, label in preprocessed_dataset_val.take(1):
    print(img.shape, label.shape)


In [ ]:
print(np.array(final_dataset))  # Shape of images
print(np.array(preprocessed_dataset_val))  # Shape of labels

In [ ]:

print(np.array(final_dataset))  # Shape of images